# DPR Reader Pipeline

## Import Statements

In [1]:
import jsonlines
import json
import text_utils
import re
from transformers import DPRReader, DPRReaderTokenizer
import pandas as pd
import numpy as np

In Transformers v4.0.0, the default path to cache downloaded models changed from '~/.cache/torch/transformers' to '~/.cache/huggingface/transformers'. Since you don't seem to have overridden and '~/.cache/torch/transformers' is a directory that exists, we're moving it to '~/.cache/huggingface/transformers' to avoid redownloading models you have already in the cache. You should only see this message once.


In [2]:
df = pd.read_csv("D:\probable_data.csv")

In [3]:
tokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
model = DPRReader.from_pretrained('facebook/dpr-reader-single-nq-base')

In [11]:
question_list = list(df['question'])
passage_list = list(df['passage'])

In [12]:
encoded_inputs = tokenizer(
        questions=question_list,
        texts=passage_list,
        return_tensors='pt',
        padding = 'max_length',
        truncation=False,
        max_length = 512
    )

outputs = model(**encoded_inputs)

In [16]:
outputs

DPRReaderOutput(start_logits=tensor([[ -7.0336,  -8.3673,  -9.1825,  ..., -12.5827, -12.5468, -12.2806],
        [ -7.0355,  -8.3648,  -9.0539,  ..., -12.3776, -12.4020, -12.3502],
        [ -5.7824,  -7.1477,  -7.9224,  ..., -11.6871, -11.7961, -11.6917],
        ...,
        [-10.9493, -11.7762, -11.7722,  ..., -12.6884, -12.6545, -12.6849],
        [ -7.6602, -10.3366, -10.0864,  ..., -12.5529, -12.5881, -12.6149],
        [ -8.4754,  -9.5537, -10.0741,  ..., -12.7039, -12.7012, -12.6917]],
       grad_fn=<ViewBackward>), end_logits=tensor([[ -5.4598,  -7.6801,  -8.2971,  ..., -11.6381, -11.7223, -11.8566],
        [ -5.5072,  -7.5863,  -8.2079,  ..., -11.8185, -11.8291, -11.8891],
        [ -4.8712,  -6.8935,  -7.5445,  ..., -10.7843, -10.7834, -10.9495],
        ...,
        [ -9.8667, -10.3519, -10.6474,  ..., -12.0380, -12.0677, -12.0355],
        [ -7.7354,  -9.5509,  -9.7739,  ..., -11.9091, -11.8976, -11.8883],
        [ -6.9224,  -7.9385,  -8.7129,  ..., -12.0161, -12.0226, 

In [13]:
tokens = tokenizer.convert_ids_to_tokens(list(encoded_inputs['input_ids'].numpy()[0]))

In [19]:
print(f"Answer not using 'titles' argument in the tokenizer\n")

for q, p in (zip(question_list, passage_list)):
    predicted_span = ''.join(p[np.argmax(outputs[0].detach().numpy()) : np.argmax(outputs[1].detach().numpy()) + 1])
    print(f"{q}")
    print(f"A) {predicted_span}\n")

Answer not using 'titles' argument in the tokenizer

which is the most common use of opt-in e-mail marketing
A) 

which is the most common use of opt-in e-mail marketing
A) 

which is the most common use of opt-in e-mail marketing
A) 

which is the most common use of opt-in e-mail marketing
A) 

which is the most common use of opt-in e-mail marketing
A) 

how i.met your mother who is the mother
A) 

how i.met your mother who is the mother
A) 

how i.met your mother who is the mother
A) 

how i.met your mother who is the mother
A) 

how i.met your mother who is the mother
A) 

what type of fertilisation takes place in humans
A) 

what type of fertilisation takes place in humans
A) 

what type of fertilisation takes place in humans
A) 

what type of fertilisation takes place in humans
A) 

what type of fertilisation takes place in humans
A) 

who had the most wins in the nfl
A) 

who had the most wins in the nfl
A) 

who had the most wins in the nfl
A) 

who had the most wins in the nfl
